In [68]:
import pandas as pd
import env
import numpy as np
import acquire_r
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [141]:
df = acquire_r.get_data_from_sql()

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
payment_type_id             7043 non-null int64
internet_service_type_id    7043 non-null int64
contract_type_id            7043 non-null int64
customer_id                 7043 non-null object
gender                      7043 non-null object
senior_citizen              7043 non-null int64
partner                     7043 non-null object
dependents                  7043 non-null object
tenure                      7043 non-null int64
phone_service               7043 non-null object
multiple_lines              7043 non-null object
online_security             7043 non-null object
online_backup               7043 non-null object
device_protection           7043 non-null object
tech_support                7043 non-null object
streaming_tv                7043 non-null object
streaming_movies            7043 non-null object
paperless_billing           7043 non-null object
monthly_charges 

In [143]:
df.isnull().sum()

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

### Looks like no values are null

In [144]:
df[['churn']].sort_values(by = 'churn')

,churn
0,No
4889,No
4888,No
4887,No
4886,No
...,...
3870,Yes
3871,Yes
3872,Yes
3858,Yes


In [145]:
df['total_charges'] = df['total_charges'].str.strip()

In [146]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [147]:
df.isnull().sum()

payment_type_id              0
internet_service_type_id     0
contract_type_id             0
customer_id                  0
gender                       0
senior_citizen               0
partner                      0
dependents                   0
tenure                       0
phone_service                0
multiple_lines               0
online_security              0
online_backup                0
device_protection            0
tech_support                 0
streaming_tv                 0
streaming_movies             0
paperless_billing            0
monthly_charges              0
total_charges               11
churn                        0
contract_type                0
internet_service_type        0
payment_type                 0
dtype: int64

In [148]:
# I am going to drop the 11 'blank' (and then converted to null) values in total_charges column
df = df.dropna()

In [149]:
# convert total_charges from str to float values:
df.total_charges = df.total_charges.astype(float)

In [150]:
df.head(2)

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.40,No,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,...,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)


In [153]:
df.churn.value_counts()

No     5163
Yes    1869
Name: churn, dtype: int64

In [139]:
df = df.replace('Yes', 1)
df = df.replace('No', 0)
df = df.replace('No internet service', 2)

In [140]:
df

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,1,0003-MKNFE,Male,0,0,0,9,1,...,0,0,1,0,59.90,542.40,0,Month-to-month,DSL,Mailed check
1,4,1,1,0013-MHZWF,Female,0,0,1,9,1,...,1,1,1,1,69.40,571.45,0,Month-to-month,DSL,Credit card (automatic)
2,1,1,1,0015-UOCOJ,Female,1,0,0,7,1,...,0,0,0,1,48.20,340.35,0,Month-to-month,DSL,Electronic check
3,1,1,1,0023-HGHWL,Male,1,0,0,1,0,...,0,0,0,1,25.10,25.10,1,Month-to-month,DSL,Electronic check
4,3,1,1,0032-PGELS,Female,0,1,1,1,0,...,0,0,0,0,30.50,30.50,1,Month-to-month,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,3,9950-MTGYX,Male,0,1,1,28,1,...,2,2,2,1,20.30,487.95,0,Two year,None,Credit card (automatic)
7039,2,3,3,9953-ZMKSM,Male,0,0,0,63,1,...,2,2,2,0,25.25,1559.30,0,Two year,None,Mailed check
7040,4,3,3,9964-WBQDJ,Female,0,1,0,71,1,...,2,2,2,1,24.40,1725.40,0,Two year,None,Credit card (automatic)
7041,3,3,3,9972-EWRJS,Female,0,1,1,67,1,...,2,2,2,1,19.25,1372.90,0,Two year,None,Bank transfer (automatic)
